# Para nuevos usuarios, se tienen que instalar por primera vez las librerías con las siguientes líneas de código:

In [2]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install webdriver_manager

  Using cached webdriver_manager-3.8.5-py2.py3-none-any.whl (27 kB)
  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


# Librerias

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import re
import numpy as np
import time
import itertools
from bs4 import BeautifulSoup
import pandas as pd
import csv
import os
import matplotlib.pyplot as plt
from time import sleep
from tqdm import tqdm

# Driver

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())
#va hacia la página principal de LinkedIn
driver.get("https://www.linkedin.com/home")
#maximiza la pantalla para evitar errores
driver.maximize_window()

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.81M/6.81M [00:05<00:00, 1.38MB/s]
C:\Users\Endeavor\AppData\Local\Temp\ipykernel_14284\1026514882.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


# Credenciales

In [4]:
#establecemos un delay
delay = 5
#Establecer ruta donde se ingresará el correo
USERID= WebDriverWait(driver,delay).until(EC.element_to_be_clickable((By.XPATH,"//div[@class='text-input flex']/input[@id='session_key']")))
#manda el correo. Cambiar por credenciales necesarias
USERID.send_keys("correo.delete.me@gmail.com")
#Establece ruta para ingresar correo
password= WebDriverWait(driver,delay).until(EC.element_to_be_clickable((By.XPATH,"//div[@class='text-input flex']/input[@id='session_password']")))
#manda la contraseña. Cambiar por credenciales necesarias
password.send_keys("#Contraseña1")
#da click en log in
log_in= WebDriverWait(driver,delay).until(EC.element_to_be_clickable((By.XPATH,"//button[@data-id='sign-in-form__submit-btn']"))).click()

# Extraer los links de un csv

#### Hay que establecer correctamente el directorio, además el archivo csv no debe tener títulos en la primera fila

In [15]:
# Establecer el directorio de trabajo
os.chdir('C:/Users/)

# Crear una lista vacía para almacenar los enlaces
contents = []

# Leer el archivo CSV y guardar la primera columna (URLs) en la lista "contents"
with open('FILE_NAME.csv', 'rt') as cp_csv:
    # Leer el contenido del archivo CSV
    cp_url = csv.reader(cp_csv)
    
    # Recorrer cada fila del CSV
    for row in cp_url:
        # Obtener el primer elemento de la fila (enlace) y guardarlo en la variable "links"
        links = row[0]
        
        # Agregar el enlace a la lista "contents"
        contents.append(links)

# Loop que hace el scraper

In [16]:
# Establecer una lista vacía para almacenar los resultados
lista_filas = []
# Iniciar el bucle, "i" es cada enlace individual, "contents" es la lista con todos los enlaces
for i in tqdm(contents) :
    # Navegar al enlace actual
    driver.get(i)
    time.sleep(2)
    
    # Intentar obtener el nombre de la compañía
    try:
        time.sleep(2)
        name_loc = driver.find_elements(By.XPATH, "//*[contains(@class, 'block mt2')]//h1[contains(@class, 'ember-view t-24 t-black t-bold')]//span[@dir='ltr']")
        name = name_loc[0].get_attribute('textContent').strip()
    # Si no se encuentra, asigna un valor NaN
    except:
        name = np.nan
        
    # Intentar obtener el número de empleados en LinkedIn
    try:
        number_employees_loc= driver.find_elements(By.XPATH,"//div[@class = 'mt1']//span[@class='link-without-visited-state t-bold t-black--light']")
        number_employees_text = number_employees_loc[0].get_attribute('textContent').strip()
        number_employees_linkedin= int(re.search(r'\d+', number_employees_text).group())
    except:
        number_employees_linkedin = np.nan
    
    # Intentar hacer clic en el botón "Acerca de"
    try:
        acerca_de_boton = WebDriverWait(driver,delay).until(EC.element_to_be_clickable((By.LINK_TEXT,"About"))).click()
        time.sleep(3)
    except:
        pass
    
    # Intentar obtener la descripción de la empresa
    try:
        resumen_find = driver.find_elements(By.XPATH, "//section[contains(@class, 'artdeco-card')]//p[contains(@class, 'text-body-small')]")
        desc = resumen_find[0].get_attribute('textContent').strip()
    except:
        desc=np.nan
        
    # Intentar obtener el sitio web de la empresa
    try:
        website_loc = driver.find_elements(By.XPATH, "//div[@class='org-grid__content-height-enforcer']//*[text()='Website']/following-sibling::dd[contains(@class, 'mb4') and contains(@class, 't-black--light') and contains(@class, 'text-body')]")
        website = website_loc[0].get_attribute('textContent').strip()
    except:
        website = np.nan
        
    # Intentar obtener el intervalo del número de empleados cuando no hay un número exacto
    try:
        intervalo_find = driver.find_elements(By.XPATH,"//dt[text()='Company size']/following-sibling::dd[1]")
        intervalo =intervalo_find[0].get_attribute('textContent').strip()
    except:
        intervalo = np.nan
    
    # Intentar obtener la industria de la empresa
    try:
        time.sleep(1)
        industry_loc = driver.find_elements(By.XPATH, "//div[@class='org-grid__content-height-enforcer']//*[text()='Industry']/following-sibling::dd[contains(@class, 'mb4') and contains(@class, 't-black--light') and contains(@class, 'text-body')]")
        industry = industry_loc[0].get_attribute('textContent').strip()
    except:
        industry = np.nan
        
    # Intentar obtener la ubicación de la empresa  
    try:
        time.sleep(1)
        location_loc = driver.find_elements(By.XPATH,"//div[@class='org-grid__content-height-enforcer']//*[text()='Headquarters']/following-sibling::dd[contains(@class, 'mb4') and contains(@class, 't-black--light') and contains(@class, 'text-body')]")
        location = location_loc[0].get_attribute('textContent').strip()
        time.sleep(3)
    except:
        location = np.nan
    
    # Intentar obtener el año de fundación de la empresa
    try:
        fundacion_loc = driver.find_elements(By.XPATH, "//div[@class='org-grid__content-height-enforcer']//*[text()='Founded']/following-sibling::dd[contains(@class, 'mb4') and contains(@class, 't-black--light') and contains(@class, 'text-body')]")
        fundacion = fundacion_loc[0].get_attribute('textContent').strip()
    except:
        fundacion = np.nan
    
    # Intentar obtener la ubicación de la empresa, pero dentro de la caja de "About"
    try:
        hq_about_find = driver.find_elements(By.XPATH, "//*[contains(@class, 'artdeco-card') and contains(@class, 'org-page-details-module__card-spacing') and contains(@class, 'org-about-module__margin-bottom')]//*[text()='Headquarters']/following-sibling::dd[contains(@class, 'mb4') and contains(@class, 't-black--light') and contains(@class, 'text-body-medium')]")
        hq_about = hq_about_find[0].get_attribute('textContent').strip()
    except:
        hq_about = np.nan
    
    # Intentar obtener el número de ubicaciones de la empresa
    try:
        number_hq_loc = driver.find_elements(By.XPATH, "//div[contains(@class, 'artdeco-card')]//h3[contains(@class, 't-bold')]")
        number_hq_text = number_hq_loc[0].get_attribute('textContent').strip()
        number_hq= int(re.search(r'\d+', number_hq_text).group())
    except:
        number_hq = np.nan
    
    # Intentar dar click en la sección de "People"
    try:
        people_button=driver.find_element(By.XPATH, "//ul[contains(@class, 'org-page-navigation__items ')]//*[contains(., 'People')]/.")
        people_button.click()
        time.sleep(2)
        show_more_button= driver.find_element(By.XPATH, "//button[@class='org-people__show-more-button t-16 t-16--open t-black--light t-bold']")
        show_more_button.click()
    # Si la empresa no tiene este botón, sigue al siguiente comando
    except:
        pass 
    
    # Cuando encuentra el botón y da click, intenta encontrar la información de los países de sus empleados
    try:
        time.sleep(2)
        country1 = driver.find_element(By.XPATH, '//div[@class="artdeco-carousel__content"]/ul[@class="artdeco-carousel__slider ember-view"]/li[@class="artdeco-carousel__item active ember-view"]')
        # Extrae el texto de la tabla y lo divide en líneas
        countries_raw=country1.get_attribute('textContent').splitlines()
        # Elimina los espacios en blanco y guarda en la variable 'countries'
        countries=[x.strip() for x in countries_raw if x.strip()] 
        # Elimina las primeras dos observaciones que no son útiles
        del countries[0:2]
        countries = [x for x in countries if x != "toggle off"]
        countries = [x.replace(',', '') for x in countries]
    except:
        countries=np.nan

    # Intentar obtener la información de las escuelas  
    try:
        people_schools_find = driver.find_element(By.XPATH, '//div[@class="artdeco-carousel__content"]/ul[@class="artdeco-carousel__slider ember-view"]/li[@data-item-index="1"]')
        people_schools_raw=people_schools_find.get_attribute('textContent').splitlines()
        people_schools= [x.strip() for x in people_schools_raw if x.strip()] 
        del people_schools[0:2]
        people_schools = [x for x in people_schools if x != "toggle off"]
        people_schools = [x.replace(',', '') for x in people_schools]
    except:
        people_schools=np.nan
        
    # Intentar obtener la información de las ocupaciones   
    try:
        time.sleep(4)
        people_occup_find =driver.find_element(By.XPATH, '//div[@class="artdeco-carousel__content"]/ul[@class="artdeco-carousel__slider ember-view"]/li[@data-item-index="2"]')
        people_occup_raw=people_occup_find.get_attribute('textContent').splitlines()
        people_occup=[x.strip() for x in people_occup_raw if x.strip()] 
        del people_occup[0]
        people_occup = [x for x in people_occup if x != "toggle off"]
        people_occup = [x.replace(',', '') for x in people_occup]
    except:
        people_occup= np.nan
        
    # Los siguientes bloques de código intentan hacer clic en el botón "Next" 
    # para navegar a través de las diferentes secciones del carrusel
    try:
        time.sleep(2)
        next_page = driver.find_element(By.XPATH, "//button[@aria-label='Next']")
        next_page.click()
    except:
        pass
    
    try:
        time.sleep(2)
        next_page = driver.find_element(By.XPATH, "//button[@aria-label='Next']")
        next_page.click()
    except:
        pass
    
    # Intentar obtener la información de las habilidades
    try:
        time.sleep(2)
        people_skills_find =driver.find_element(By.XPATH, '//div[@class="artdeco-carousel__content"]/ul[@class="artdeco-carousel__slider ember-view"]/li[@data-item-index="3"]')
        people_skills_raw=people_skills_find.get_attribute('textContent').splitlines()
        people_skills= [x.strip() for x in people_skills_raw if x.strip()] 
        del people_skills[0:2]
        people_skills = [x for x in people_skills if x != "toggle off"]
        people_skills = [x.replace(',', '') for x in people_skills]
    except:
        people_skills = np.nan

    # Intentar obtener la información de las carreras
    try:
        time.sleep(2)
        people_majors_find =driver.find_element(By.XPATH, '//div[@class="artdeco-carousel__content"]/ul[@class="artdeco-carousel__slider ember-view"]/li[@data-item-index="4"]')
        people_majors_raw=people_majors_find.get_attribute('textContent').splitlines()
        people_majors= [x.strip() for x in people_majors_raw if x.strip()] 
        del people_majors[0:2]
        people_majors = [x for x in people_majors if x != "toggle off"]
        people_majors = [x.replace(',', '') for x in people_majors]
    except:
        people_majors=np.nan
    
        
    # Le da el nombre a cada variable de la fila    
    fila = pd.DataFrame({'Company Name': [name], 'Industry': [industry],'Location':[location], 'HQ About':[hq_about],
                         '# of HQ':[number_hq],'# of Employees on LinkedIn':[number_employees_linkedin],
                         "# of Employees (interval)":[intervalo], 'Description':[desc], 'Website':[website] , 
                         'Founding Year': [fundacion],'Countries':[countries],'Schools':[people_schools],
                         'Occupations':[people_occup],'Skills':[people_skills],'Majors':[people_majors]})
    
    # Añade la fila actual (con los datos de la empresa) a la lista 'lista_filas'
    lista_filas.append(fila)
    
    # Concatena todas las filas en la lista 'lista_filas' para crear el DataFrame final 'df'
    df = pd.concat(lista_filas).fillna('')

100%|██████████████████████████████████████████████████████████████████████████████| 213/213 [1:45:13<00:00, 29.64s/it]


In [17]:
# Esta línea de código sirve para ver si la información extraída es correcta
# Debería tener la información y no tener valores NaN en la columna 'Countries' hacia adelante
# No es preocupante si hay valores NaN en la columna 'HQ About'
df

,Company Name,Industry,Location,HQ About,# of HQ,# of Employees on LinkedIn,# of Employees (interval),Description,Website,Founding Year,Countries,Schools,Occupations,Skills,Majors
0,STUCK?,Education,,,,8.0,2-10 employees,"STUCK is a live, on-demand technology platform...",https://stuckapp.co,2022,"[2 Egypt, 1 Krenglbach, 1 Saudi Arabia, 1 Uppe...","[4 University of Oxford, 2 Alexandria Universi...","[4 Information Technology, 4 Education, 3 Busi...","[4 Public Speaking, 4 Teaching, 4 Research Ski...","[3 Teacher Education Multiple Levels, 3 Univer..."
0,THIQAH,IT Services and IT Consulting,"Al Sahafah Dist, Riyadh",,2.0,1.0,"501-1,000 employees","With THIQAH comes the power of technology, ena...",http://www.thiqah.sa,2012,"[912 Saudi Arabia, 624 Riyadh Saudi Arabia, 21...","[218 King Saud University, 80 Al-Imam Muhammad...","[221 Engineering, 207 Information Technology, ...","[389 Microsoft Office, 387 Teamwork, 330 Leade...","[169 Computer Science, 155 Computational Scien..."
0,MRSOOL | مرسول,"Technology, Information and Internet","Riyadh, RIYADH",,2.0,1.0,201-500 employees,"Established in 2015, Mrsool grew rapidly over ...",https://mrsool.co,2015,"[1336 Saudi Arabia, 555 Riyadh Saudi Arabia, 3...","[38 King Saud University, 34 Imam Muhammad ibn...","[242 Operations, 169 Customer Success and Supp...","[229 Customer Service, 184 Communication, 172 ...","[36 Computer Science, 30 Computational Science..."
0,Dsquares,"Technology, Information and Internet",,,4.0,221.0,51-200 employees,Dsquares specializes in offering loyalty progr...,https://www.dsquares.com/,2012,"[171 Egypt, 90 Cairo Egypt, 47 Al Jizah Egypt,...","[30 Cairo University, 21 Ain Shams University,...","[57 Engineering, 44 Business Development, 36 O...","[67 Microsoft Office, 59 Teamwork, 56 Leadersh...","[32 Computer Science, 26 Computational Science..."
0,Soum,Internet Marketplace Platforms,,,,61.0,11-50 employees,Soum is on a mission to reimagine recommerce i...,http://info.soum.sa,,"[43 Saudi Arabia, 35 Riyadh Saudi Arabia, 13 R...",[11 King Fahd University of Petroleum & Minera...,"[12 Operations, 12 Engineering, 8 Business Dev...","[26 Microsoft Office, 21 Teamwork, 19 Leadersh...","[6 Industrial Engineering, 5 Computer Science,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,ZeeSofts,"Technology, Information and Internet","Riyadh, RUH",,1.0,3.0,2-10 employees,We offer creative Designs and Strive for innov...,http://www.zeesofts.com,2012,"[1 Saudi Arabia, 1 Pakistan, 1 Tamil Nadu Indi...","[1 University of Northern Iowa, 1 University o...","[1 Business Development, 1 Community and Socia...","[1 Python (Programming Language), 1 System Adm...","[1 Computer Science, 1 Computational Science]"
0,MIS - Al Moammar Information Systems Co.,IT Services and IT Consulting,"Riyadh, Al Sahafa Dist.",,3.0,571.0,"501-1,000 employees","""Empowered Integrators""​\n\nSince its formatio...",http://www.mis.com.sa,1979,"[510 Saudi Arabia, 284 Riyadh Saudi Arabia, 13...",[21 Al-Imam Muhammad Ibn Saud Islamic Universi...,"[243 Information Technology, 70 Sales, 69 Prog...","[139 Microsoft Office, 127 Project Management,...","[73 Computer Science, 61 Computational Science..."
0,ToYou | تويو,"Technology, Information and Internet",,,1.0,389.0,"501-1,000 employees","ToYou provides fast and convenient shopping, f...",https://toyou.io/ar,,"[268 Saudi Arabia, 191 Riyadh Saudi Arabia, 92...",[16 Princess Nourah Bint Abdulrahman Universit...,"[66 Operations, 59 Information Technology, 54 ...","[122 Teamwork, 112 Microsoft Office, 106 Custo...",[21 Business Administration and Management Gen...
0,MENAbytes,"Technology, Information and Internet",,,,2.0,2-10 employees,Covering tech and startups from the Middle Eas...,http://www.menabytes.com,2017,"[1 Pakistan, 1 Cairo Egypt, 1 Punjab Pakistan,...","[1 University Of Central Punjab, 1 The German ...","[1 Information Technology, 1 Media and Communi...","[1 Email Marketing, 1 Copy Editing, 1 Brand Aw...","[1 Organizational Leaders

# Wrangling

In [18]:
# Combinar las columnas 'Location' y 'HQ About', utilizando 'Location' 
# como fuente principal y completando los valores faltantes con 'HQ About'
df['Location'] = df['Location'].combine_first(df['HQ About'])

#------------------------------------------------------------------------------

# Definir una función para extraer los primeros cinco países y sus números
def extract_first_five(countries_list):
    first_five = [x.replace("toggle off, ", "") for x in countries_list[:5]]
    return first_five

# Aplicar la función a la columna 'Countries'
df['first_five'] = df['Countries'].apply(extract_first_five)

# Crear columnas separadas para cada país y su respectivo recuento
for i in range(1, 6):
    df[f'N{i}_Country'] = df['first_five'].apply(lambda x: int(x[i - 1].split(' ')[0]) if len(x) >= i else None)
    df[f'Country{i}'] = df['first_five'].apply(lambda x: ' '.join(x[i - 1].split(' ')[1:]) if len(x) >= i else None)

# Eliminar la columna temporal 'first_five'
df.drop(columns=['first_five'], inplace=True)

#------------------------------------------------------------------------------

# Definir una función para extraer las primeros cinco escuelas y sus números
def extract_first_five_schools(schools_list):
    first_five = schools_list[:5]
    return first_five

# Aplicar la función a la columna 'Schools'
df['first_five_schools'] = df['Schools'].apply(extract_first_five_schools)

# Crear columnas separadas para cada escuela y su respectivo recuento
for i in range(1, 6):
    df[f'N{i}_School'] = df['first_five_schools'].apply(lambda x: int(x[i - 1].split(' ')[0]) if len(x) >= i else None)
    df[f'School{i}'] = df['first_five_schools'].apply(lambda x: ' '.join(x[i - 1].split(' ')[1:]) if len(x) >= i else None)

# Eliminar la columna temporal 'first_five'
df.drop(columns=['first_five_schools'], inplace=True)

#----------------------------------------------------------------------------

# Definir una función para extraer las primeros cinco ocupaciones y sus números
def extract_first_five_occupations(occupations_list):
    first_five = occupations_list[:5]
    return first_five

# Aplicar la función a la columna 'Occupations'
df['first_five_occupations'] = df['Occupations'].apply(extract_first_five_occupations)

# Crear columnas separadas para cada ocupación y su respectivo recuento
for i in range(1, 6):
    df[f'N{i}_Occupation'] = df['first_five_occupations'].apply(lambda x: int(x[i - 1].split(' ')[0]) if len(x) >= i else None)
    df[f'Occupation{i}'] = df['first_five_occupations'].apply(lambda x: ' '.join(x[i - 1].split(' ')[1:]) if len(x) >= i else None)

# Eliminar la columna temporal 'first_five'
df.drop(columns=['first_five_occupations'], inplace=True)



#----------------------------------------------------------------------------

# Definir una función para extraer las primeros cinco habilidades y sus números
def extract_first_five_skills(skills_list):
    first_five = skills_list[:5]
    return first_five

# Aplicar la función a la columna 'Skills'
df['first_five_skills'] = df['Skills'].apply(extract_first_five_schools)

# Crear columnas separadas para cada habilidad y su respectivo recuento
for i in range(1, 6):
    df[f'N{i}_Skill'] = df['first_five_skills'].apply(lambda x: int(x[i - 1].split(' ')[0]) if len(x) >= i else None)
    df[f'Skill{i}'] = df['first_five_skills'].apply(lambda x: ' '.join(x[i - 1].split(' ')[1:]) if len(x) >= i else None)

# Eliminar la columna temporal 'first_five'
df.drop(columns=['first_five_skills'], inplace=True)

#-----------------------------------------------------------------------------

# Definir una función para extraer las primeros cinco carreras y sus números
def extract_first_five_majors(majors_list):
    first_five = majors_list[:5]
    return first_five

# Aplicar la función a la columna 'Majors'
df['first_five_majors'] = df['Majors'].apply(extract_first_five_majors)

# Crear columnas separadas para cada carrera y su respectivo recuento
for i in range(1, 6):
    df[f'N{i}_Major'] = df['first_five_majors'].apply(lambda x: int(x[i - 1].split(' ')[0]) if len(x) >= i else None)
    df[f'Major{i}'] = df['first_five_majors'].apply(lambda x: ' '.join(x[i - 1].split(' ')[1:]) if len(x) >= i else None)

# Eliminar la columna temporal 'first_five'
df.drop(columns=['first_five_majors'], inplace=True)

# Eliminar las columnas 'HQ About', 'Countries', 'Schools', 'Occupations', 'Skills' y 'Majors', 
# creando un nuevo DataFrame llamado 'df_clean'
df_clean = df.drop(columns=['HQ About','Countries','Schools','Occupations','Skills','Majors'])



# Data Frame final (clean)

# Este Data Frame incluye:
- Información básica de las empresas (Industria, Ubicación, Website, Año de Fundación
## Información de los empleados:
- Dónde viven
- Dónde estudiaron
- Qué estudiaron
- Qué hacen dentro de la empresa
- Qué habilidades tienen

In [19]:
# Establecer esta configuración para desplegar todas las columnas
# Sirve para verificar que el proceso anterior haya sido exitoso
pd.set_option('display.max_columns', 1000)
# Despliega el data frame final
df_clean

,Company Name,Industry,Location,# of HQ,# of Employees on LinkedIn,# of Employees (interval),Description,Website,Founding Year,N1_Country,Country1,N2_Country,Country2,N3_Country,Country3,N4_Country,Country4,N5_Country,Country5,N1_School,School1,N2_School,School2,N3_School,School3,N4_School,School4,N5_School,School5,N1_Occupation,Occupation1,N2_Occupation,Occupation2,N3_Occupation,Occupation3,N4_Occupation,Occupation4,N5_Occupation,Occupation5,N1_Skill,Skill1,N2_Skill,Skill2,N3_Skill,Skill3,N4_Skill,Skill4,N5_Skill,Skill5,N1_Major,Major1,N2_Major,Major2,N3_Major,Major3,N4_Major,Major4,N5_Major,Major5
0,STUCK?,Education,,,8.0,2-10 employees,"STUCK is a live, on-demand technology platform...",https://stuckapp.co,2022,2.0,Egypt,1.0,Krenglbach,1.0,Saudi Arabia,1.0,Upper Austria Austria,1.0,United Kingdom,4.0,University of Oxford,2.0,Alexandria University,1.0,Herzen State Pedagogical University of Russia,1.0,Colby College,1.0,Udacity,4.0,Information Technology,4.0,Education,3.0,Business Development,2.0,Media and Communication,1.0,Engineering,4.0,Public Speaking,4.0,Teaching,4.0,Research Skills,4.0,English as a Second Language (ESL),4.0,Curriculum Development,3.0,Teacher Education Multiple Levels,3.0,University Studies,2.0,Computer Science,2.0,Applied Linguistics,2.0,English Language and Literature General
0,THIQAH,IT Services and IT Consulting,"Al Sahafah Dist, Riyadh",2.0,1.0,"501-1,000 employees","With THIQAH comes the power of technology, ena...",http://www.thiqah.sa,2012,912.0,Saudi Arabia,624.0,Riyadh Saudi Arabia,219.0,Riyadh Region,90.0,Egypt,59.0,Cairo Egypt,218.0,King Saud University,80.0,Al-Imam Muhammad Ibn Saud Islamic University,72.0,Princess Nourah Bint Abdulrahman University,36.0,Udacity,30.0,King AbdulAziz University,221.0,Engineering,207.0,Information Technology,160.0,Business Development,119.0,Operations,87.0,Program and Project Management,389.0,Microsoft Office,387.0,Teamwork,330.0,Leadership,278.0,Project Management,260.0,Management,169.0,Computer Science,155.0,Computational Science,130.0,Information Technology,87.0,Business Administration and Management General,55.0,Computer Software Engineering
0,MRSOOL | مرسول,"Technology, Information and Internet","Riyadh, RIYADH",2.0,1.0,201-500 employees,"Established in 2015, Mrsool grew rapidly over ...",https://mrsool.co,2015,1336.0,Saudi Arabia,555.0,Riyadh Saudi Arabia,391.0,Riyadh Region,379.0,Makkah Saudi Arabia,334.0,Makkah Region,38.0,King Saud University,34.0,Imam Muhammad ibn Saud Islamic University,32.0,King AbdulAziz University,32.0,Princess Nourah Bint Abdulrahman University,26.0,Ain Shams University,242.0,Operations,169.0,Customer Success and Support,110.0,Sales,62.0,Business Development,53.0,Administrative,229.0,Customer Service,184.0,Communication,172.0,Teamwork,171.0,Microsoft Office,166.0,Leadership,36.0,Computer Science,30.0,Computational Science,28.0,Business Administration and Management General,26.0,English Language and Literature General,18.0,Accounting and Related Services
0,Dsquares,"Technology, Information and Internet",,4.0,221.0,51-200 employees,Dsquares specializes in offering loyalty progr...,https://www.dsquares.com/,2012,171.0,Egypt,90.0,Cairo Egypt,47.0,Al Jizah Egypt,35.0,Giza,12.0,United States,30.0,Cairo University,21.0,Ain Shams University,14.0,Helwan University Cairo,10.0,Information Technology Institute (ITI),8.0,Udacity,57.0,Engineering,44.0,Business Development,36.0,Operations,20.0,Information Technology,20.0,Sales,67.0,Microsoft Office,59.0,Teamwork,56.0,Leadership,52.0,Team Leadership,51.0,Management,32.0,Computer Science,26.0,Computational Science,19.0,Business Administration and Management General,11.0,Information Technology,10.0,Marketing
0,Soum,Internet Marketplace Platforms,,,61.0,11-50 employees,Soum is on a mission to reimagine recommerce i...,http://info.soum.sa,,43.0,Saudi Arabia,35.0,Riyadh Saudi Arabia,13.0,Riyadh Region,9.0,Egypt,4.0,Cairo Egypt,11.0,King Fahd University of Petroleum & Minerals -...,7.0,Udacity,5.0,Prin

# Guardar el CSV

In [20]:
# Cambiar 'file_name' por el nombre de archivo deseado
df_clean.to_csv('CLEAN_DF_NAME.csv', index=False)

In [21]:
# cerrar el driver
driver.quit()